In [ ]:
import json
import os

import compmusic.dunya.conn
from compmusic.dunya.docserver import get_document_as_json, get_mp3

import urllib


In [ ]:
# paste your Dunya token below
token = 'XXX'
compmusic.dunya.conn.set_token(token)


In [ ]:
# paths
data_folder = './data/'

symbtr_mbid_url = 'https://raw.githubusercontent.com/MTG/SymbTr/master/symbTr_mbid.json'
symbtr_mbid = json.loads(urllib.urlopen(symbtr_mbid_url).read())

makam_recs_file = './dataset.json'
makam_recs = json.load(open(makam_recs_file))


In [ ]:
for makam, recs in makam_recs.items():
    # makam folder
    makam_folder = os.path.join(data_folder, makam.title())
    
    # create makam directory if it doesn't exist
    if not os.path.exists(makam_folder):
        os.makedirs(makam_folder)
    
    for rec in recs:
        recid = os.path.split(rec)[-1]
        print makam + ', ' + recid
        
        # audio metadata
        metadata = get_document_as_json(recid, 'audioanalysis', 'metadata')
        
        ## ------------------ Obtain the relevant SymbTr score ------------
        # get the symbtr work
        assert len(metadata['works']) == 1, 'More that one work in the recording %s' % recid
        workid = metadata['works'][0]['mbid']

        # get the symbtr name
        symbtr = None
        for sm in symbtr_mbid:
            if workid in sm['uuid']:
                symbtr = sm['name']

        assert symbtr is not None, 'No SymbTr score matched for the work %s' % workid

        # create symbtr folder
        symbtr_folder = os.path.join(makam_folder, symbtr)
        if not os.path.exists(symbtr_folder):
            os.makedirs(symbtr_folder)
            
        # copy the relevant SymbTr documents
        # TODO

        ## ------------------ Obtain the relevant recording and features ------------
        # create audio folder
        rec_folder = os.path.join(symbtr_folder, recid)
        if not os.path.exists(rec_folder):
            os.makedirs(rec_folder)
        
        # metadata 
        metadata_file = os.path.join(rec_folder, 'metadata.json')
        json.dump(metadata, open(metadata_file, 'w'), indent=4)
        
        # pitch
        pitch = get_document_as_json(recid, 'audioanalysis', 'pitch_filtered')
        pitch_file = os.path.join(rec_folder, 'pitch_filtered.json')
        json.dump(pitch, open(pitch_file, 'w'))

        # mp3
        mp3 = get_mp3(recid)
        mp3_file = os.path.join(rec_folder, recid + '.mp3')
        open(mp3_file, 'w').write(mp3)

        # pitch distribution
        pd = get_document_as_json(recid, 'audioanalysis', 'pitch_distribution')
        pd_file = os.path.join(rec_folder, 'pitch_distribution.json')
        json.dump(pd, open(pd_file, 'w'))

        # stable notes
        nm = get_document_as_json(recid, 'audioanalysis', 'note_models')
        nm_file = os.path.join(rec_folder, 'note_models.json')
        json.dump(nm, open(nm_file, 'w'))
        